In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# Huobi

In [ ]:
from ccstabilizer import Exchange
from ccstabilizer import secrets

In [ ]:
# export
import contextlib, requests
from decimal import Decimal, ROUND_DOWN


class Huobi(Exchange):

    def __init__(self):
        super().__init__()
        self.huobi = HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY'])
        self.account_id = self.huobi.get_accounts()['data'][0]['id']
        self.update_trading_specifications()

    def update_trading_specification(self, crypto_symbol, fiat_symbol):
        return self.update_trading_specifications().get((crypto_symbol, fiat_symbol), {})

    def update_trading_specifications(self):
        specs = self.huobi.get_symbols()['data']
        for spec in specs:
            crypto_symbol, fiat_symbol = spec.get('base-currency', '').upper(), spec.get('quote-currency', '').upper()
            trading_spec = self.trading_specifications.setdefault((crypto_symbol, fiat_symbol), {})
            trading_spec['min_trade_unit'] = max(
                Decimal(repr(spec.get('limit-order-min-order-amt', 0.0001))),
                Decimal(repr(spec.get('sell-market-min-order-amt', 0.0001))),
                Decimal('0.1') ** Decimal(repr(spec.get('amount-precision', 0))),
            )
            trading_spec['min_trade_fiat_money_limit'] = Decimal(repr(spec.get('min-order-value', 1)))
            trading_spec['fee_rate'] = Decimal('0.002')
            trading_spec['liquid'] = spec.get('api-trading', 'disabled') == 'enabled'
        return self.trading_specifications

    def get_portfolio(self):
        return {
            asset['currency'].upper(): Decimal(asset.get('balance', '0')) for asset in self.huobi.get_balance(**{'account-id': self.account_id})['data']['list'] \
                if asset['type'] == 'trade' and Decimal(asset.get('balance', '0')) > 0
        }

    def get_price(self, crypto_symbol, fiat_symbol):
        now_ticker = self.huobi.get_ticker(symbol=f'{crypto_symbol.lower()}{fiat_symbol.lower()}')['tick']
        now_buy_fiat_price_without_fee = Decimal(now_ticker.get('ask', ['Infinity', '0'])[0])
        now_sell_fiat_price_without_fee = Decimal(now_ticker.get('bid', ['0', '0'])[0])
        fee_rate = self.get_trading_specification(crypto_symbol, fiat_symbol).get('fee_rate', 0)
        return {
            'now_buy_fiat_price': self.get_buy_fiat_price(now_buy_fiat_price_without_fee, fee_rate),
            'now_sell_fiat_price': self.get_sell_fiat_price(now_sell_fiat_price_without_fee, fee_rate),
            'now_buy_fiat_price_without_fee': now_buy_fiat_price_without_fee,
            'now_sell_fiat_price_without_fee': now_sell_fiat_price_without_fee,
        }

    def get_buy_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee / (1 - fee_rate)

    def get_sell_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee * (1 - fee_rate)

    def buy(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        if amount <= 0:
            raise Exception('Huobi::buy  amount <= 0')
        fiat_money = (fiat_price_without_fee * amount).quantize(Decimal('1.00'), ROUND_DOWN)
        self.huobi.send_order(
            **{'account-id': self.account_id, 'symbol': f'{crypto_symbol.lower()}{fiat_symbol.lower()}', 'type': 'buy-market', 'amount': str(fiat_money)}
        )

    def sell(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        if amount <= 0:
            raise Exception('Huobi::sell  amount <= 0')
        self.huobi.send_order(
            **{'account-id': self.account_id, 'symbol': f'{crypto_symbol.lower()}{fiat_symbol.lower()}', 'type': 'sell-market', 'amount': str(amount)}
        )

    def has_enough_unused_fiat_money(self, buy_fiat_money, unused_fiat_money):
        return buy_fiat_money <= unused_fiat_money

    def trade_fiat_money_is_larger_than_limit(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        return fiat_price_without_fee * amount > self.get_trading_specification(crypto_symbol, fiat_symbol).get('min_trade_fiat_money_limit', 0)

## Huobi API
* https://github.com/HuobiRDCenter/huobi_Python
* https://huobiapi.github.io/docs

* https://github.com/chiangqiqi/pyhuobi/blob/master/huobi/utils.py
* https://github.com/MJeremy2017/HuobiApi/blob/master/huobiApi/service.py

In [ ]:
# export
import base64
import contextlib
import hashlib
import hmac
import json
import urllib
import urllib.parse
import urllib.request
import requests

from datetime import datetime


class HuobiSVC(object):

    methods = {
        # Public methods
        'get_kline':           {'url': 'market/history/kline', 'method': 'GET', 'private': False},
        'get_depth':           {'url': 'market/depth', 'method': 'GET', 'private': False},
        'get_trade':           {'url': 'market/trade', 'method': 'GET', 'private': False},
        'get_tickers':         {'url': 'market/tickers', 'method': 'GET', 'private': False},
        'get_ticker':          {'url': 'market/detail/merged', 'method': 'GET', 'private': False},
        'get_detail':          {'url': 'market/detail', 'method': 'GET', 'private': False},
        'get_symbols':         {'url': 'v1/common/symbols', 'method': 'GET', 'private': False},
        'get_currencies':      {'url': 'v1/common/currencys', 'method': 'GET', 'private': False},
        # Private methods
        'get_accounts':        {'url': 'v1/account/accounts', 'method': 'GET', 'private': True},
        'get_balance':         {'url': 'v1/account/accounts/{account-id}/balance', 'method': 'GET', 'private': True},
        'send_order':          {'url': 'v1/order/orders/place', 'method': 'POST', 'private': True},
        'cancel_order':        {'url': 'v1/order/orders/{order-id}/submitcancel', 'method': 'POST', 'private': True},
        'order_info':          {'url': 'v1/order/orders/{order-id}', 'method': 'GET', 'private': True},
        'order_matchresults':  {'url': 'v1/order/orders/{order-id}/matchresults', 'method': 'GET', 'private': True},
        'orders_list':         {'url': 'v1/order/orders', 'method': 'GET', 'private': True},
        'orders_matchresults': {'url': 'v1/order/matchresults', 'method': 'GET', 'private': True},
        'open_orders':         {'url': 'v1/order/openOrders', 'method': 'GET', 'private': True},
        'cancel_open_orders':  {'url': 'v1/order/orders/batchCancelOpenOrders', 'method': 'POST', 'private': True},
        'withdraw':            {'url': 'v1/dw/withdraw/api/create', 'method': 'POST', 'private': True},
        'cancel_withdraw':     {'url': 'v1/dw/withdraw-virtual/{withdraw-id}/cancel', 'method': 'POST', 'private': True},
    }

    RETRY_INTERVAL = 60

    def __init__(self, access_key, secret_key):
        self.access_key = access_key
        self.secret_key = secret_key
        self.base_url = 'https://api-aws.huobi.pro/'

    def __getattr__(self, command):
        def call_api(**kwargs):

            while True:
                try:

                    url = self.base_url + type(self).methods[command]['url'].format(**kwargs)

                    payload = kwargs
                    headers = {}

                    payload_str = urllib.parse.urlencode(payload)
                    if self.methods[command]['private']:
                        if self.methods[command]['method'] == 'GET':
                            return self.api_key_get(url, **kwargs)
                        else:
                            return self.api_key_post(url, **kwargs)
                    else:
                        if self.methods[command]['method'] == 'GET':
                            return self.http_get_request(url, **kwargs)

                except Exception as e:
                    raise
                    time.sleep(type(self).RETRY_INTERVAL)

        return call_api

    def http_get_request(self, url, add_to_headers=None, **params):
        headers = {
            'Content-type': 'application/x-www-form-urlencoded',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
        }
        if add_to_headers:
            headers.update(add_to_headers)
        postdata = urllib.parse.urlencode(params)
        with contextlib.closing(
            requests.get(url, postdata, headers=headers, timeout=5)
        ) as response:
            try:
                response_json = response.json()
                if response_json['status'] == 'ok':
                    return response_json
                else:
                    raise Exception('status == error')
            except BaseException as e:
                print(f'httpGet failed, detail: {response.text}, {e}')
                print(f'url = {url}')
                print(f'params = {params}')
                raise

    def http_post_request(self, url, add_to_headers=None, **params):
        headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
        if add_to_headers:
            headers.update(add_to_headers)
        postdata = json.dumps(params)
        with contextlib.closing(
            requests.post(url, postdata, headers=headers, timeout=10)
        ) as response:
            try:
                response_json = response.json()
                if response_json['status'] == 'ok':
                    return response_json
                else:
                    raise Exception('status == error')
            except BaseException as e:
                print(f'httpPost failed, detail: {response.text}, {e}')
                print(f'url = {url}')
                print(f'params = {params}')
                raise

    def api_key_get(self, url, **params):
        method = 'GET'
        timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
        params.update({'AccessKeyId': self.access_key,
                       'SignatureMethod': 'HmacSHA256',
                       'SignatureVersion': '2',
                       'Timestamp': timestamp})

        host_name = urllib.parse.urlparse(url).hostname
        host_name = host_name.lower()
        request_path = urllib.parse.urlparse(url).path
        params['Signature'] = self.createSign(params, method, host_name, request_path, self.secret_key)

        return self.http_get_request(url, **params)

    def api_key_post(self, url, **params):
        method = 'POST'
        timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
        params_to_sign = {'AccessKeyId': self.access_key,
                          'SignatureMethod': 'HmacSHA256',
                          'SignatureVersion': '2',
                          'Timestamp': timestamp}

        host_name = urllib.parse.urlparse(url).hostname
        host_name = host_name.lower()
        request_path = urllib.parse.urlparse(url).path
        params_to_sign['Signature'] = self.createSign(params_to_sign, method, host_name, request_path, self.secret_key)
        url += '?' + urllib.parse.urlencode(params_to_sign)
        return self.http_post_request(url, **params)

    def createSign(self, pParams, method, host_url, request_path, secret_key):
        sorted_params = sorted(pParams.items(), key=lambda d: d[0], reverse=False)
        encode_params = urllib.parse.urlencode(sorted_params)
        payload = [method, host_url, request_path, encode_params]
        payload = '\n'.join(payload)
        payload = payload.encode(encoding='UTF8')
        secret_key = secret_key.encode(encoding='UTF8')

        digest = hmac.new(secret_key, payload, digestmod=hashlib.sha256).digest()
        signature = base64.b64encode(digest)
        signature = signature.decode()
        return signature

    # get KLine
#     def get_kline(self, symbol, period, size=150):
#         """
#         :param symbol: btcusdt, ethbtc, ...
#         :param period: 可选值：{1min, 5min, 15min, 30min, 60min, 1day, 1mon, 1week, 1year }
#         :param size: 可选值： [1,2000]
#         :return:
#         """
#         params = {'symbol': symbol,
#                   'period': period,
#                   'size': size}

#         url = self.market_url + '/market/history/kline'
#         return self.http_get_request(url, params)

    # get market prices
    def get_kline_df(self, symbol, period, size):
        res = self.get_kline(symbol=symbol, period=period, size=size)
        if res['status'] == 'ok':
            import pandas as pd
            kline_df = pd.DataFrame(res['data'])
            return kline_df
        else:
            raise Exception('Query failed with status: {}'.format(res))

    # get market depth
#     def get_depth(self, symbol, type):
#         """
#         :param symbol
#         :param type: 可选值：{ percent10, step0, step1, step2, step3, step4, step5 }
#         :return:
#         """
#         params = {'symbol': symbol,
#                   'type': type}

#         url = self.market_url + '/market/depth'
#         return self.http_get_request(url, params)

    # get trade detail
#     def get_trade(self, symbol):
#         """
#         :param symbol
#         :return:
#         """
#         params = {'symbol': symbol}

#         url = self.market_url + '/market/trade'
#         return self.http_get_request(url, params)

    # Tickers detail
#     def get_tickers(self):
#         """
#         :return:
#         """
#         params = {}
#         url = self.market_url + '/market/tickers'
#         return self.http_get_request(url, params)

    # get merge ticker
#     def get_ticker(self, symbol):
#         """
#         :param symbol:
#         :return:
#         """
#         params = {'symbol': symbol}

#         url = self.market_url + '/market/detail/merged'
#         return self.http_get_request(url, params)

    # get Market Detail 24 hour volume
#     def get_detail(self, symbol):
#         """
#         :param symbol
#         :return:
#         """
#         params = {'symbol': symbol}

#         url = self.market_url + '/market/detail'
#         return self.http_get_request(url, params)

    # get available symbols
#     def get_symbols(self, long_polling=None):
#         """
#         """
#         params = {}
#         if long_polling:
#             params['long-polling'] = long_polling
#         path = '/v1/common/symbols'
#         return self.api_key_get(params, path)

    # Get available currencies
#     def get_currencies(self):
#         """
#         :return:
#         """
#         params = {}
#         url = self.market_url + '/v1/common/currencys'

#         return self.http_get_request(url, params)

    # Get all the trading assets
#     def get_trading_assets(self):
#         """
#         :return:
#         """
#         params = {}
#         url = self.market_url + '/v1/common/symbols'

#         return self.http_get_request(url, params)

    """
    Trade/Account API
    """

#     def get_accounts(self):
#         """
#         :return:
#         """
#         path = "/v1/account/accounts"
#         params = {}
#         return self.api_key_get(params, path)

    # get account balance
#     def get_balance(self, account_id=None):
#         """
#         :param account_id
#         :return:
#         """

#         if account_id is None:
#             accounts = self.get_accounts()
#             account_id = accounts['data'][0]['id']

#         url = f'{self.base_url}v1/account/accounts/{account_id}/balance'
#         params = {'account-id': account_id}
#         return self.api_key_get(url, **params)

    # get balance for a currency
    def get_balance_currency(self, account_id, symbol):
        res = self.get_balance(account_id=account_id)
        if res['status'] == 'ok':
            res_dict = {}
            import pandas as pd
            balance_df = pd.DataFrame(res['data']['list'])
            res_df = balance_df[balance_df['currency'] == symbol]
            res_dict['trade_balance'] = float(res_df[res_df['type'] == 'trade']['balance'].values[0])
            res_dict['frozen_balance'] = float(res_df[res_df['type'] == 'frozen']['balance'].values[0])
            return res_dict
        else:
            raise Exception('Query failed with status -> {}'.format(res['status']))

    # Making Orders
#     def send_order(self, acct_id, amount, source, symbol, _type, price=0, stop_price=0, operator=None):
#         """
#         :param acct_id: account id
#         :param amount:
#         :param source: 如果使用借贷资产交易，请在下单接口的请求参数source中填写'margin-api'
#         :param symbol:
#         :param _type: options {buy-market：市价买, sell-market：市价卖, buy-limit：限价买, sell-limit：限价卖}
#         :param price:
#         :param stop_price:
#         :param operator: gte – greater than and equal (>=), lte – less than and equal (<=)
#         :return:
#         """

#         params = {"account-id": acct_id,
#                   "amount": amount,
#                   "symbol": symbol,
#                   "type": _type,
#                   "source": source}
#         if price:
#             params["price"] = price
#         if stop_price:
#             params["stop-price"] = stop_price
#         if operator:
#             params["operator"] = operator

#         url = '/v1/order/orders/place'
#         return self.api_key_post(params, url)

    # cancel an order
#     def cancel_order(self, order_id):
#         """
#         :param order_id:
#         :return:
#         """
#         params = {}
#         url = f'{self.base_url}v1/order/orders/{order_id}/submitcancel'
#         return self.api_key_post(url, **params)

    # get an order info
#     def order_info(self, order_id):
#         """
#         :param order_id:
#         :return:
#         """
#         params = {}
#         url = f'{self.base_url}v1/order/orders/{order_id}'
#         return self.api_key_get(url, **params)

    # get order results
#     def order_matchresults(self, order_id):
#         """
#         :param order_id:
#         :return:
#         """
#         params = {}
#         url = f'{self.base_url}v1/order/orders/{order_id}/matchresults'
#         return self.api_key_get(url, **params)

    # get order list
#     def orders_list(self, symbol, states, types=None, start_date=None, end_date=None, _from=None, direct=None, size=None):
#         """
#         :param symbol:
#         :param states: options {pre-submitted 准备提交, submitted 已提交, partial-filled 部分成交, partial-canceled 部分成交撤销, filled 完全成交, canceled 已撤销}
#         :param types: options {buy-market：市价买, sell-market：市价卖, buy-limit：限价买, sell-limit：限价卖}
#         :param start_date:
#         :param end_date:
#         :param _from:
#         :param direct: options {prev 向前，next 向后}
#         :param size:
#         :return:
#         """
#         params = {'symbol': symbol,
#                   'states': states}

#         if types:
#             params['types'] = types
#         if start_date:
#             params['start-date'] = start_date
#         if end_date:
#             params['end-date'] = end_date
#         if _from:
#             params['from'] = _from
#         if direct:
#             params['direct'] = direct
#         if size:
#             params['size'] = size
#         url = '/v1/order/orders'
#         return self.api_key_get(params, url)

    # get matched orders
#     def orders_matchresults(self, symbol, types=None, start_date=None, end_date=None, _from=None, direct=None, size=None):
#         """
#         :param symbol:
#         :param types: options {buy-market：市价买, sell-market：市价卖, buy-limit：限价买, sell-limit：限价卖}
#         :param start_date:
#         :param end_date:
#         :param _from:
#         :param direct: options {prev 向前，next 向后}
#         :param size:
#         :return:
#         """
#         params = {'symbol': symbol}

#         if types:
#             params['types'] = types
#         if start_date:
#             params['start-date'] = start_date
#         if end_date:
#             params['end-date'] = end_date
#         if _from:
#             params['from'] = _from
#         if direct:
#             params['direct'] = direct
#         if size:
#             params['size'] = size
#         url = '/v1/order/matchresults'
#         return self.api_key_get(params, url)

    # get open orders
#     def open_orders(self, account_id, symbol, side='', size=10):
#         """
#         :param symbol:
#         :return:
#         """
#         params = {}
#         url = "/v1/order/openOrders"
#         if symbol:
#             params['symbol'] = symbol
#         if account_id:
#             params['account-id'] = account_id
#         if side:
#             params['side'] = side
#         if size:
#             params['size'] = size

#         return self.api_key_get(params, url)

    # batch cancel orders
#     def cancel_open_orders(self, account_id, symbol, side='', size=10):
#         """
#         :param symbol:
#         :return:
#         """
#         params = {}
#         url = "/v1/order/orders/batchCancelOpenOrders"
#         if symbol:
#             params['symbol'] = symbol
#         if account_id:
#             params['account-id'] = account_id
#         if side:
#             params['side'] = side
#         if size:
#             params['size'] = size

#         return self.api_key_post(params, url)

    # withdraw currencies
#     def withdraw(self, address, amount, currency, fee=0, addr_tag=""):
#         """
#         :param address_id:
#         :param amount:
#         :param currency:btc, ltc, bcc, eth, etc ...(火币Pro支持的币种)
#         :param fee:
#         :param addr-tag:
#         :return: {
#                   "status": "ok",
#                   "data": 700
#                 }
#         """
#         params = {'address': address,
#                   'amount': amount,
#                   "currency": currency,
#                   "fee": fee,
#                   "addr-tag": addr_tag}
#         url = '/v1/dw/withdraw/api/create'

#         return self.api_key_post(params, url)

    # cancel withdraw order
#     def cancel_withdraw(self, address_id):
#         """
#         :param address_id:
#         :return: {
#                   "status": "ok",
#                   "data": 700
#                 }
#         """
#         params = {}
#         url = '/v1/dw/withdraw-virtual/{0}/cancel'.format(address_id)

#         return self.api_key_post(params, url)

    """
    MARGIN API
    """

    # create and send margin order
#     def send_margin_order(self, account_id, amount, symbol, _type, price=0):
#         """
#         :param account_id:
#         :param amount:
#         :param symbol:
#         :param _type: options {buy-market：市价买, sell-market：市价卖, buy-limit：限价买, sell-limit：限价卖}
#         :param price:
#         :return:
#         """
#         try:
#             accounts = self.get_accounts()
#             acct_id = accounts['data'][0]['id']
#         except BaseException as e:
#             print('get acct_id error.%s' % e)
#             acct_id = account_id

#         params = {"account-id": acct_id,
#                   "amount": amount,
#                   "symbol": symbol,
#                   "type": _type,
#                   "source": 'margin-api'}
#         if price:
#             params["price"] = price

#         url = '/v1/order/orders/place'
#         return self.api_key_post(params, url)

    # exchange account to margin account
#     def exchange_to_margin(self, symbol, currency, amount):
#         """
#         :param amount:
#         :param currency:
#         :param symbol:
#         :return:
#         """
#         params = {"symbol": symbol,
#                   "currency": currency,
#                   "amount": amount}

#         url = "/v1/dw/transfer-in/margin"
#         return self.api_key_post(params, url)

    # margin account to exchange account
#     def margin_to_exchange(self, symbol, currency, amount):
#         """
#         :param amount:
#         :param currency:
#         :param symbol:
#         :return:
#         """
#         params = {"symbol": symbol,
#                   "currency": currency,
#                   "amount": amount}

#         url = "/v1/dw/transfer-out/margin"
#         return self.api_key_post(params, url)

    # get margin
#     def get_margin(self, symbol, currency, amount):
#         """
#         :param amount:
#         :param currency:
#         :param symbol:
#         :return:
#         """
#         params = {"symbol": symbol,
#                   "currency": currency,
#                   "amount": amount}
#         url = "/v1/margin/orders"
#         return self.api_key_post(params, url)

    # repay
#     def repay_margin(self, order_id, amount):
#         """
#         :param order_id:
#         :param amount:
#         :return:
#         """
#         params = {"order-id": order_id,
#                   "amount": amount}
#         url = "/v1/margin/orders/{0}/repay".format(order_id)
#         return self.api_key_post(params, url)

    # loan order
#     def loan_orders(self, symbol, currency, start_date="", end_date="", start="", direct="", size=""):
#         """
#         :param symbol:
#         :param currency:
#         :param direct: prev 向前，next 向后
#         :return:
#         """
#         params = {"symbol": symbol,
#                   "currency": currency}
#         if start_date:
#             params["start-date"] = start_date
#         if end_date:
#             params["end-date"] = end_date
#         if start:
#             params["from"] = start
#         if direct and direct in ["prev", "next"]:
#             params["direct"] = direct
#         if size:
#             params["size"] = size
#         url = "/v1/margin/loan-orders"
#         return self.api_key_get(params, url)

    # get margin balance
#     def margin_balance(self, symbol):
#         """
#         :param symbol:
#         :return:
#         """
#         params = {}
#         url = "/v1/margin/accounts/balance"
#         if symbol:
#             params['symbol'] = symbol

#         return self.api_key_get(params, url)

#     def margin_loan_info(self, symbol):
#         """
#         :param symbol:
#         :return:
#         """
#         params = {}
#         url = "/v1/margin/loan-info"
#         if symbol:
#             params['symbol'] = symbol
#         return self.api_key_get(params, url)

In [ ]:
import pprint
for spec in HuobiSVC('', '').get_symbols()['data']:
    if spec['base-currency'] == 'ar' and spec['quote-currency'] == 'usdt':
        pprint.PrettyPrinter(indent=4).pprint(spec)

{   'amount-precision': 2,
    'api-trading': 'enabled',
    'base-currency': 'ar',
    'buy-market-max-order-value': 100000,
    'limit-order-max-buy-amt': 330000,
    'limit-order-max-order-amt': 330000,
    'limit-order-max-sell-amt': 330000,
    'limit-order-min-order-amt': 0.001,
    'max-order-amt': 330000,
    'min-order-amt': 0.001,
    'min-order-value': 5,
    'price-precision': 4,
    'quote-currency': 'usdt',
    'sell-market-max-order-amt': 33000,
    'sell-market-min-order-amt': 0.001,
    'state': 'online',
    'symbol': 'arusdt',
    'symbol-partition': 'innovation',
    'value-precision': 8}


In [ ]:
Huobi().get_trading_specification('AR', 'ETH')

{'min_trade_unit': Decimal('0.001'),
 'min_trade_fiat_money_limit': Decimal('0.01'),
 'fee_rate': Decimal('0.002'),
 'liquid': True}

In [ ]:
HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).get_accounts()

{'status': 'ok',
 'data': [{'id': 23069350, 'type': 'spot', 'subtype': '', 'state': 'working'}]}

In [ ]:
HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).get_balance(**{'account-id': 23069350})

In [ ]:
HuobiSVC('', '').get_ticker(symbol='ethusdt')

{'status': 'ok',
 'ch': 'market.ethusdt.detail.merged',
 'ts': 1617943956046,
 'tick': {'amount': 330127.5631112884,
  'open': 2017.15,
  'close': 2073.45,
  'high': 2097.91,
  'id': 244600985546,
  'count': 667328,
  'low': 1983.85,
  'version': 244600985546,
  'ask': [2073.46, 2.0973],
  'vol': 675589798.6738303,
  'bid': [2073.45, 53.7823]}}

In [ ]:
# HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).send_order(**{'account-id': 23069350, 'symbol': 'xrpusdt', 'type': 'buy-limit', 'amount': 10, 'price': 0.5})

{'status': 'ok', 'data': '251846879621754'}

In [ ]:
# HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).send_order(**{'account-id': 23069350, 'symbol': 'arusdt', 'type': 'buy-market', 'amount': '5'})

{'status': 'ok', 'data': '251963306317075'}

In [ ]:
# HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).send_order(**{'account-id': 23069350, 'symbol': 'arusdt', 'type': 'sell-market', 'amount': '0.16'})

{'status': 'ok', 'data': '251963314036695'}

In [ ]:
HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).orders_list(**{'symbol': 'xrpusdt', 'states': 'submitted'})

{'status': 'ok',
 'data': [{'id': 251846879621754,
   'symbol': 'xrpusdt',
   'account-id': 23069350,
   'client-order-id': '',
   'amount': '10.000000000000000000',
   'price': '0.500000000000000000',
   'created-at': 1617946728705,
   'type': 'buy-limit',
   'field-amount': '0.0',
   'field-cash-amount': '0.0',
   'field-fees': '0.0',
   'finished-at': 0,
   'source': 'spot-api',
   'state': 'submitted',
   'canceled-at': 0}]}

In [ ]:
HuobiSVC(os.environ['HUOBI_API_ACCESS_KEY'], os.environ['HUOBI_API_SECRET_KEY']).order_info(**{'order-id': 251846879621754})

{'status': 'ok',
 'data': {'id': 251846879621754,
  'symbol': 'xrpusdt',
  'account-id': 23069350,
  'client-order-id': '',
  'amount': '10.000000000000000000',
  'price': '0.500000000000000000',
  'created-at': 1617946728705,
  'type': 'buy-limit',
  'field-amount': '0.0',
  'field-cash-amount': '0.0',
  'field-fees': '0.0',
  'finished-at': 0,
  'source': 'spot-api',
  'state': 'submitted',
  'canceled-at': 0}}